## LDA with raw Python

* **Collapsed Gibbs Sampling** (Steyvers & Griffiths, 2007)
* **Online Variational Bayes** (Hoffman, Blei & Bach, 2010)

NB: a demo with randomly generated toy data, but it clearly shows the code works.

In [87]:
import numpy as np

In [93]:
# Helpers

class Indexer(object):
    def __init__(self):
        self.objs_to_ints = {}
        self.ints_to_objs = {}
    def __repr__(self):
        return str([str(self.get_object(i)) for i in xrange(0, len(self))])
    def __len__(self):
        return len(self.objs_to_ints)
    def get_object(self, index):
        if (index not in self.ints_to_objs):
            return None
        else:
            return self.ints_to_objs[index]
    def contains(self, object):
        return self.index_of(object) != -1
    def index_of(self, object):
        if (object not in self.objs_to_ints):
            return -1
        else:
            return self.objs_to_ints[object]
    def get_index(self, object, add=True):
        if not add:
            return self.index_of(object)
        if (object not in self.objs_to_ints):
            new_idx = len(self.objs_to_ints)
            self.objs_to_ints[object] = new_idx
            self.ints_to_objs[new_idx] = object
        return self.objs_to_ints[object]
    
def normalize(arr):
    if type(arr)==list: arr = np.array(arr)
    return arr / arr.sum()

def safe_log(x, minval=0.0000000001):
    return np.log(x.clip(min=minval))

$\large{\textbf{Collapsed Gibbs}}$

In [94]:
# Generate toy set

num_topics = 3
topic2words = {0:['cat','dog','horse','pig','lion'],
               1:['house','church','room','kitchen','table'],
               2:['art','painting','paris','brush','dance']}

word_indexer = Indexer()

def sample_document(topic_mix, size=100):
    doc = []
    for i in range(size):
        topic = np.random.choice(range(num_topics),p=topic_mix)
        rand_topic = np.random.randint(0,num_topics)
        word = np.random.choice(topic2words[topic])
        w_id = word_indexer.get_index(word)
        doc.append((w_id,rand_topic))
    return doc

num_docs = 100
docid2doc = defaultdict(list)
cut1, cut2, cut3 = 25, 50, 75 
for d in range(num_docs):
    if d<cut1:
        docid2doc[d] = sample_document([1,0,0])
    elif d<cut2:
        docid2doc[d] = sample_document([0,1,0])
    elif d<cut3:
        docid2doc[d] = sample_document([0,0,1])
    else:
        docid2doc[d] = sample_document([1/3.,1/3.,1/3.])

In [ ]:
W, D, T = len(word_indexer), len(docid2doc), num_topics

C_WT, C_DT = np.zeros((W,T)), np.zeros((D,T))

for docid in docid2doc.iterkeys():
    for w_id,topic in docid2doc[docid]:
        C_WT[w_id,topic] += 1
        C_DT[docid,topic] += 1

beta, alpha = .1, .1
beta_vec = np.array([beta]*T); W_beta = W*beta_vec
alpha_vec = np.array([alpha]*T); T_alpha = T*alpha_vec

def gibbs_sample(w_i, d): # TODO: need to offset by priors!
    a = (C_WT[w_i,:] + beta_vec) / (C_WT.sum(axis=0) + W_beta)
    b = (C_DT[d,:] + alpha_vec) / (C_DT[d,:].sum() + T_alpha)
    c = a * b
    p_z = normalize(c)
    return np.random.choice(np.arange(T), p=p_z)

def document_loglikelihood(d):
    a = C_WT / C_WT.sum(axis=0)
    b = C_DT[d,:] / C_DT[d,:].sum()
    m = a * b
    return safe_log(m.sum(axis=1)).sum() # log(W|d)

def predict(topic, k=5): # get top k words given topic
    return [word_indexer.get_object(w_id) for w_id in np.argsort(C_WT[:,topic])[::-1][:k]]

def gibbs_loop(n_iter=10, n_print=1):
    for e in range(n_iter):
        for docid in docid2doc.iterkeys(): # for each doc
            for w_id,topic in docid2doc[docid]: # for each entry
                if C_WT[w_id,topic]==0 or C_DT[docid,topic]==0:
                    continue
                C_WT[w_id,topic] -= 1
                C_DT[docid,topic] -= 1
                new_topic = gibbs_sample(w_id,docid)
                C_WT[w_id,new_topic] += 1
                C_DT[docid,new_topic] += 1
        if e % n_print == 0:
            print repr(document_loglikelihood(0))

In [90]:
print "Before"
for topic in range(T):
    print predict(topic)
print
print "Training"
gibbs_loop()
print
print "After"
for topic in range(T):
    print predict(topic)

Before
['art', 'church', 'table', 'painting', 'brush']
['lion', 'pig', 'painting', 'art', 'kitchen']
['horse', 'room', 'dog', 'pig', 'art']

Training
-40.639427155306606
-41.597639333589107
-138.62467082903808
-138.72268162918883
-138.84041909431863
-138.92251789131294
-138.95760565243043
-139.01725207565858
-139.07326292865298
-139.08876405935905

After
['table', 'church', 'house', 'kitchen', 'room']
['brush', 'painting', 'art', 'dance', 'paris']
['horse', 'pig', 'lion', 'dog', 'cat']


$\large{\textbf{Online Variational Bayes}}$

In [118]:
from __future__ import division

import time
import random
import numpy as np

from sklearn.preprocessing import normalize
from scipy.special import gammaln
from scipy.special import psi

In [167]:
# Generate toy set (different format from Gibbs)

num_topics = 3
topic2words = {0:['cat','dog','horse','pig','lion'],
               1:['house','church','room','kitchen','table'],
               2:['art','painting','paris','brush','dance']}

def sample_document(topic_mix, size=100):
    doc = np.zeros(len(word_indexer))
    for i in range(size):
        topic = np.random.choice(range(num_topics),p=topic_mix)
        word = np.random.choice(topic2words[topic])
        w_id = word_indexer.get_index(word)
        doc[w_id] += 1
    return doc

num_docs = 100
docs = []
cut1, cut2, cut3 = 25, 50, 75 
for d in range(num_docs):
    if d<cut1:
        docs.append(sample_document([1,0,0]))
    elif d<cut2:
        docs.append(sample_document([0,1,0]))
    elif d<cut3:
        docs.append(sample_document([0,0,1]))
    else:
        docs.append(sample_document([1/3.,1/3.,1/3.]))  
docs = np.array(docs)
print '#docs x vocab-size = ' + repr(docs.shape)

#docs x vocab-size = (100, 15)


In [176]:
D = num_docs
K = num_topics
W = len(word_indexer)

alpha = 0.1
eta   = 0.01
tau0  = 1.0
kappa = 0.5

def rho(t):
    return np.power(tau0 + t, -kappa)

alpha_v = np.array([alpha]*K)[:,np.newaxis] # (K,1)
eta_m   = np.zeros((K, W))
eta_m.fill(eta)

l4 = gammaln(K*alpha) - K*gammaln(alpha) + (gammaln(W*eta) - W*gammaln(eta))/D

lambda_m = np.random.gamma(100, 1/100, (K, W)) # dist'n over words by topic

def predict(topic, k=5):
    w_ids = np.argsort(lambda_m[topic]/lambda_m[topic].sum())[::-1][:k]
    return [word_indexer.get_object(w_id) for w_id in w_ids]

print 'Before'
for topic in range(K):
    print predict(topic)
print
print 'Training'
print

num_epochs    = 5
num_iters     = 5
epsilon       = 1e-3
docidxs       = np.arange(D)
n_phi_batch_m = np.zeros((K, W))
batch_size    = 10
t             = 0 # time step
for e in range(num_epochs):
    
    start = time.time()
    L     = 0 # ELBO
    
    random.shuffle(docidxs)
    for d in docidxs:
        n_d   = docs[d] # (W,)
        shift = 0.0
        # E-step
        gamma_d_v = np.random.gamma(100, 1/100, (K, 1))
        for i in range(num_iters):
            # update phi: (K,W), posteriors over topics
            Eq_logtheta_d = psi(gamma_d_v) - psi(gamma_d_v.sum()) # (K,1)
            Eq_logbeta    = psi(lambda_m) - psi(lambda_m.sum(axis=1)[:,np.newaxis]) # (K,W)
            phi_d_m       = np.exp(Eq_logtheta_d + Eq_logbeta) # (K,W)
            phi_d_m       = normalize(phi_d_m, 'l1', axis=0) # posterior over *topics*
            # update gamma: (K,1), priors over topics
            n_phi_d_m      = phi_d_m * n_d # (K,W)
            gamma_d_v_next = alpha_v + n_phi_d_m.sum(axis=1)[:,np.newaxis] # (K,1)
            # calculate shift to decide if break
            shift = (1/K)*np.abs(gamma_d_v - gamma_d_v_next).sum()
            if shift < epsilon:
                break
            gamma_d_v = gamma_d_v_next
        n_phi_batch_m += n_phi_d_m
        # M-step
        if d % batch_size == 0:
            t             += 1 # 1 change every batch
            lambda_hat_m  = eta_m + (D/batch_size)*n_phi_batch_m
            lambda_m      = (1-rho(t))*lambda_m + rho(t)*lambda_hat_m
            n_phi_batch_m = np.zeros((K, W))
        # compute ELBO for current doc
        l1 = (n_d*(phi_d_m*(Eq_logtheta_d+Eq_logbeta-safe_log(phi_d_m))).sum(axis=0)).sum()
        l2 = -gammaln(gamma_d_v.sum())+((alpha_v-gamma_d_v)*Eq_logtheta_d+gammaln(gamma_d_v)).sum()
        l3 = (-gammaln(lambda_m.sum(axis=1))+((eta_m-lambda_m)*Eq_logbeta+gammaln(lambda_m)).sum(axis=1)).sum()/D
        l = l1 + l2 + l3 + l4
        L += l
    
    print 'Epoch ' + repr(e+1) + ' (time elapsed: ' + "%.2f secs" % (time.time()-start) + ')'
    print '   ELBO = ' + repr(L)
    print '   Pepl = ' + repr(np.exp(-L/docs.sum()))

print
print 'After'
for topic in range(K):
    print predict(topic)

Before
['pig', 'dance', 'house', 'table', 'lion']
['brush', 'lion', 'dance', 'room', 'cat']
['cat', 'pig', 'dog', 'art', 'house']

Training

Epoch 1 (time elapsed: 0.04 secs)
   ELBO = -22320.556990169218
   Pepl = 9.3190034686573497
Epoch 2 (time elapsed: 0.02 secs)
   ELBO = -19752.731574365778
   Pepl = 7.2085884644568878
Epoch 3 (time elapsed: 0.02 secs)
   ELBO = -19499.143547714884
   Pepl = 7.0280856328126617
Epoch 4 (time elapsed: 0.02 secs)
   ELBO = -19394.395871931058
   Pepl = 6.9548522900106517
Epoch 5 (time elapsed: 0.03 secs)
   ELBO = -19331.323506937573
   Pepl = 6.9111244377147623

After
['kitchen', 'house', 'table', 'room', 'church']
['dance', 'paris', 'brush', 'art', 'painting']
['cat', 'lion', 'pig', 'dog', 'horse']
